## Ich steh auf Femboys

In [ ]:
# applio_installer_and_launcher.py
# Paste this into a notebook cell and run.
# It runs the official Applio installer and then launches Applio,
# streaming stdout/stderr back to the notebook in real time.

import os
import sys
import subprocess
import threading
import time
from pathlib import Path

REPO_URL = "https://github.com/IAHispano/Applio.git"
REPO_DIR = "Applio"
RUN_INSTALL = True   # set False if you already ran the install and just want to launch
SKIP_APPLIO_LAUNCH = False  # set True to stop after install

def stream_process(cmd, cwd=None, env=None):
    """
    Run a command (list) and stream combined stdout/stderr line-by-line.
    Returns the process's returncode.
    """
    print(f"\n>>> RUNNING: {' '.join(cmd)}  (cwd={cwd})\n")
    # Ensure binary mode text streaming
    p = subprocess.Popen(cmd, cwd=cwd, env=env,
                         stdout=subprocess.PIPE, stderr=subprocess.STDOUT,
                         bufsize=1, text=True, universal_newlines=True)

    try:
        for line in p.stdout:
            # flush each line to the output cell immediately
            print(line.rstrip())
    except Exception as e:
        print(f"[streaming error] {e}")
    finally:
        p.wait()
        rc = p.returncode
        print(f"\n<<< PROCESS EXIT CODE: {rc}\n")
        return rc

def ensure_git_clone():
    if Path(REPO_DIR).exists():
        print(f"Repo folder '{REPO_DIR}' already exists; skipping clone.")
        return 0
    cmd = ["git", "clone", REPO_URL, REPO_DIR]
    return stream_process(cmd)

def run_repo_install():
    # prefer running the repo's provided install script so it sets up venv / ffmpeg / requirements
    installer_sh = Path(REPO_DIR) / "run-install.sh"
    if not installer_sh.exists():
        print("Installer script 'run-install.sh' not found in repo; aborting install step.")
        return 1

    # Make sure the script is executable; use bash to run it
    cmd = ["bash", str(installer_sh)]
    return stream_process(cmd, cwd=REPO_DIR)

def detect_torch_gpu(venv_python=None):
    """
    Run a small Python snippet in the venv (or current python) to detect torch + GPU/hip info.
    Returns tuple (ok:boolean, output_str).
    """
    py = venv_python or sys.executable
    check = (
        "import sys, json\n"
        "info = {'python': sys.executable}\n"
        "try:\n"
        "  import torch\n"
        "  info['torch_version'] = getattr(torch,'__version__',None)\n"
        "  info['cuda_available'] = torch.cuda.is_available() if hasattr(torch,'cuda') else False\n"
        "  info['has_hip'] = hasattr(torch.version,'hip') and torch.version.hip is not None\n"
        "except Exception as e:\n"
        "  info['torch_error'] = str(e)\n"
        "print(json.dumps(info))\n"
    )
    cmd = [py, "-c", check]
    try:
        out = subprocess.check_output(cmd, text=True, stderr=subprocess.STDOUT)
        return True, out.strip()
    except subprocess.CalledProcessError as e:
        return False, (e.output or str(e))

def run_applio():
    launcher = Path(REPO_DIR) / "run-applio.sh"
    if not launcher.exists():
        # fallback: some setups may prefer `python app.py`
        fallback = Path(REPO_DIR) / "app.py"
        if fallback.exists():
            print("run-applio.sh missing; falling back to 'python app.py'")
            return stream_process([sys.executable, str(fallback)], cwd=REPO_DIR)
        else:
            print("No run-applio.sh or app.py found to launch Applio.")
            return 1
    return stream_process(["bash", str(launcher)], cwd=REPO_DIR)

def main():
    print("Starting Applio installer+launcher helper...\n")
    # 1) clone repo if needed
    rc = ensure_git_clone()
    if rc != 0:
        print("git clone failed. If the environment blocks git, please upload the repository into the notebook filesystem.")
        # proceed anyway in case repo already present / partial
    # 2) run repo installer (streams output)
    if RUN_INSTALL:
        rc = run_repo_install()
        if rc != 0:
            print("Installer exited with non-zero code. Continue at your own risk.")
    else:
        print("RUN_INSTALL is False; skipping installer step as requested.")

    # 3) detect torch / GPU in the venv if present
    venv_py = None
    venv_path = Path(REPO_DIR) / ".venv" / "bin" / "python"
    if venv_path.exists():
        venv_py = str(venv_path)
        ok, out = detect_torch_gpu(venv_python=venv_py)
    else:
        # fall back to current python
        ok, out = detect_torch_gpu()
    print("\n=== PyTorch / GPU probe ===")
    if ok:
        print(out)
        if "'cuda_available': true" in out.lower() or '"cuda_available": true' in out:
            print("\nGood news: CUDA-enabled PyTorch detected. That usually means NVIDIA GPU support.")
        else:
            print("\nNo CUDA available (or PyTorch not GPU-enabled). If you have an AMD GPU and want training, you typically need a ROCm-built PyTorch / special wheels. See the notes in your notebook.")
    else:
        print("Could not run torch probe. Output below:")
        print(out)

    if SKIP_APPLIO_LAUNCH:
        print("SKIP_APPLIO_LAUNCH is True; exiting now.")
        return

    # 4) launch Applio (streams live)
    rc = run_applio()
    if rc != 0:
        print("Applio launcher exited with non-zero code. Check the logs above for errors.")

if __name__ == "__main__":
    main()


# Manual Executions:

In [ ]:
import subprocess
import threading
import sys
import os
import shlex

def stream_process(command):
    """
    Runs a shell command and streams stdout/stderr live.
    Blocks until process finishes.
    """
    print(f"\n>>> Running: {command}\n")

    # Start subprocess
    process = subprocess.Popen(
        command,
        shell=True,
        stdout=subprocess.PIPE,
        stderr=subprocess.STDOUT,
        text=True,
        bufsize=1
    )

    # Stream output live
    try:
        for line in process.stdout:
            print(line.rstrip())
    except Exception as e:
        print(f"[Streaming error]: {e}")

    process.wait()
    print(f"\n<<< Exit code: {process.returncode}\n")
    return process.returncode


def interactive_loop():
    print("Interactive Subprocess Shell")
    print("Type commands to execute.")
    print("Type 'exit' or 'quit' to stop.\n")

    while True:
        try:
            command = input("cmd> ").strip()
        except (EOFError, KeyboardInterrupt):
            print("\nExiting.")
            break

        if not command:
            continue

        if command.lower() in ["exit", "quit"]:
            print("Goodbye.")
            break

        stream_process(command)


if __name__ == "__main__":
    interactive_loop()
